# Desafio 1 - Pedro Lucas Barrera a1801

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.metrics import classification_report

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

## Punto 1: 
 Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [4]:
#Tomamos 5 documentos al azar
np.random.seed(42)  # Para reproducibilidad
indices = np.random.choice(X_train.shape[0], 5, replace=False)
random_items = X_train[indices]


In [29]:
print(f"total de documentos en el dataset: {X_train.shape[0]}")
print(f"dimensionalidad del vector TF-IDF: {X_train.shape[1]}")
print(f"indices de documentos seleccionados: {indices}\n")

# Para cada documento seleccionado, busco los 5 más similares
indexes_most_similar_documents = {}
for i, doc_index in enumerate(indices):
    print(f"documento {i+1} (numero: {doc_index})")
    
    current_doc = X_train[doc_index]
    similarities = cosine_similarity(current_doc, X_train).flatten()
    
    #Tomamos 6 elementos con mayor simulitud, dado que el primero sera el mismo documento (con similaridad = 1.0), y lo removemos el primero.
    most_similar_indices = similarities.argsort()[-6:][::-1]
    most_similar_indices = most_similar_indices[1:]
    indexes_most_similar_documents[i] = most_similar_indices
    
    print(f"Etiqueta del documento a comparar: {newsgroups_train.target_names[newsgroups_train.target[doc_index]]}")
    print(f"Contenido del documento original:\n {newsgroups_train.data[doc_index]}")
    print("\ntop 5 documentos mas similares:")
    
    for rank, similar_index in enumerate(most_similar_indices, 1):
        similarity_score = similarities[similar_index]
        similar_label = newsgroups_train.target_names[newsgroups_train.target[similar_index]]
        
        print(f"### {rank}°: Documento numero {similar_index}")
        print(f"- similaridad: {similarity_score:.4f}")
        print(f"- etiqueta: {similar_label}")
        print(f"- contenido:\n {newsgroups_train.data[similar_index]}")
        print()
    print("\n------------------------------------------------------------------------------------")

total de documentos en el dataset: 11314
dimensionalidad del vector TF-IDF: 101631
indices de documentos seleccionados: [7492 3546 5582 4793 3813]

documento 1 (numero: 7492)
Etiqueta del documento a comparar: comp.sys.mac.hardware
Contenido del documento original:
 Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward" 

top 5 documentos mas similares:
### 1°: Documento numero 10935
- similaridad: 0.6665
- etiqueta: comp.sys.mac.hardware
- contenido:
 Hey everybody:

   I want to buy a mac and I want to get a good price...who doesn't?  So,
could anyone out there who has found a really good deal on a Centris 650
send me the price.  I don't want to know where, unless it is mail order or
areound cleveland, Ohio.  Also, should I buy now or wait for the Power PC.

Tha

### Analizo la similitud tomando en cuenta su titulo y contenido:
- Documento 1:

    Casi todos los titulos de los documentos mas similares son de computacion, al igual con el que se lo compara. Sin embargo, hay uno que es 'misc.forsale', osea, miscelaneo a la venta. Analizando los contenidos, se ve que el documento elegido al azar es muy corto y la mayor parte de este esta ocupado por el pie de pagina: 
     
    "Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward" "

    De esta manera, la similitud puede darse por elementos puntuales como compartir algun termino o palabra mas que por una coincidencia fuerte de contenido. 

    Puede verse que en los primeros 2 mas simialres, el pie es muy parecido. luego, se comparten ciertas palabras o elementos similares entre todos, como la misma institucion de email (@po.cwru.edu).

- Documento 2:

    Casi todos los títulos de los documentos más similares son de computación y hardware, al igual que el documento que se compara. Todos pertenecen a categorías relacionadas con sistemas de computación, específicamente comp.sys.ibm.pc.hardware para cuatro de los cinco documentos similares, mientras que el documento original pertenece a comp.os.ms-windows.misc.

    El documento elegido al azar trata sobre software de backup (CPBackup, Fastback) y menciona DMA access y tape drive. Los documentos más similares comparten un dominio técnico común centrado en hardware de computación y controladores de datos. El término "DMA" aparece en todos los documentos similares, junto con otros términos técnicos específicos como "SCSI", "floppy controller", "busmastering" e "ISA". Ademas, todos los documentos tratan sobre hardware de computación, específicamente controladores y transferencias de datos. Se mencionan especificaciones técnicas como límites de 16MB y canales DMA, así como sistemas operativos como DOS y Windows. Los protocolos técnicos como ASPI e ISA también aparecen en varios documentos, mostrando un contexto de conocimiento técnico compartido.

    En este caso, al ser el documento original mas extenso se puede ver como la similitud entre los mismos pareciera basarse mas que en el 1 en el contenido semantico mas que en palabras o terminos comunes como el pie de pagina.

- Documento 3:

    Casi todos los títulos de los documentos más similares pertenecen a la categoría misc.forsale, al igual que el documento que se compara. Tres de los cinco documentos similares pertenecen a esta categoría, mientras que dos pertenecen a comp.graphics (gráficos por computadora).

    El documento elegido al azar trata sobre hardware de computación en venta, específicamente un drive de 5.25", monitor monocromático y motherboard 8088. Analizando los contenidos, se ve que con los 3 documentos más similares se comparte exactamente la misma firma personal: "Libertarian, atheist, semi-anarchal Techno-Rat." Esta firma única, combinada con el contenido de hardware, crea similitudes artificialmente altas.

    De esta manera, la similitud puede darse por elementos puntuales como compartir la firma personal más que por una coincidencia fuerte de contenido temático. Luego, se comparten ciertas palabras o elementos similares entre todos, como términos de hardware de computación (motherboard, monitor, drive).

    En este caso, al ser el documento original muy corto y contener una firma personal única, se puede ver como la similitud entre los mismos es causada en gran parte por elementos únicos (firma personal), pero tambien un poco por contenido temático real (hardware de computación), mostrando un patrón híbrido entre el documento 1 (solo elementos únicos) y el documento 2 (solo contenido temático) (aunque mas cercano al documento 1)




- Documento 4:

    Casi todos los títulos de los documentos más similares pertenecen a categorías relacionadas con política y gobierno, al igual que el documento que se compara. El documento original pertenece a talk.politics.guns, mientras que los similares incluyen talk.politics.guns, sci.crypt, talk.politics.misc y alt.atheism.

    El documento elegido al azar trata sobre políticas de control de armas en parques nacionales de Canadá y menciona temas como protección personal, osos y psicópatas. Analizando los contenidos, se ve que el documento más similar es un comunicado oficial de la Casa Blanca sobre el incidente de Waco, que trata sobre control de armas y operaciones federales. El segundo documento trata sobre criptografía y el programa Clipper, que también involucra temas de privacidad y control gubernamental.

    De esta manera, la similitud puede darse por elementos temáticos relacionados con control gubernamental y políticas públicas más que por palabras específicas. Puede verse que en los primeros documentos más similares, se comparten temas como control de armas, operaciones federales y políticas de seguridad. Luego, se comparten ciertas palabras o elementos similares entre todos, como términos políticos ("government", "federal", "control") y referencias a instituciones gubernamentales.

    En este caso, al ser el segundo documento original más extenso y tratar sobre políticas públicas, se puede ver como la similitud entre los mismos se basa en contenido temático real relacionado con control gubernamental y políticas de seguridad, mostrando una coincidencia semántica mayor que los documentos anteriores.

- Documento 5:
 
    Casi todos los títulos de los documentos más similares pertenecen a categorías relacionadas con deportes y entretenimiento, al igual que el documento que se compara. El documento original pertenece a rec.sport.baseball, mientras que los similares incluyen rec.sport.baseball, rec.sport.hockey, rec.autos y rec.motorcycles.

    El documento elegido al azar trata sobre béisbol y estadísticas de jugadores, mencionando temas como bateo, carreras y jugadores específicos. Analizando los contenidos, se ve que el documento más similar trata sobre béisbol y estadísticas similares, mientras que el segundo documento trata sobre hockey, mostrando una similitud temática en deportes. Los siguientes documentos tratan sobre automóviles y motocicletas, que aunque son diferentes deportes, comparten el contexto de entretenimiento y aficiones.

    De esta manera, la similitud puede darse por elementos temáticos relacionados con deportes y entretenimiento más que por palabras específicas. Puede verse que en los primeros documentos más similares, se comparten temas como estadísticas deportivas, jugadores y competiciones. Luego, se comparten ciertas palabras o elementos similares entre todos, como términos deportivos ("game", "team", "player") y referencias a entretenimiento.
    
    Este es el documento original más extenso y tratar sobre deportes específicos, por lo que puede verse que caemos en un caso similar al del documento 4, donde se puede ver como la similitud entre los mismos se basa en contenido temático real relacionado con deportes y entretenimiento, mostrando una coincidencia semántica consistente con el dominio de los documentos.


Algo a destacar es que para el documento 1, la similitud fue la mas alta, pero principalmente debido a lo corto del documento y que compartian una firma en comun, lo cual en realidad no hace mucho al contenido real del mismo sino mas a una gigante coincidencia de palabras exactas. De esta forma, puede verse la improtancia de generar preprocesamientos sobre los documentos para sacar secciones que puedan generar confusiones y falsas similitudes si lo que se busca es evaluar el contenido tematico.

## Punto 2: 
  Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

Analizo un poco la estructura del dataset

In [17]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

print(f"shape del conjunto de train: {X_train.shape}")
print(f"shape del conjunto de test: {X_test.shape}")
print(f"numero de clases: {len(newsgroups_train.target_names)}")
print(f"clases: {newsgroups_train.target_names}")

shape del conjunto de train: (11314, 101631)
shape del conjunto de test: (7532, 101631)
numero de clases: 20
clases: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Pruebo primero los resultados con hiperparametros default

In [18]:
# Función para evaluar un modelo
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    
    return f1, y_pred



multinomial_nb = MultinomialNB()
f1_multinomial, _ = evaluate_model(multinomial_nb, X_train, y_train, X_test, y_test)
print(f"F1 Score multinomial default: {f1_multinomial}")

complement_nb = ComplementNB()
f1_complement, _ = evaluate_model(complement_nb, X_train, y_train, X_test, y_test)
print(f"F1 Score multinomial default: {f1_multinomial}")

F1 Score multinomial default: 0.5854345727938506
F1 Score multinomial default: 0.5854345727938506


Optimizo ahora los hiperparametros del vectorizador y de los clasificadores, probando distintas combinaciones entre ellos y quedandome con la mejor

In [19]:

# Lista de configuraciones de vectorizadores para probar
vectorizer_configs = [
    {
        'name': 'TF-IDF default',
        'params': {}
    },
    {
        'name': 'TF-IDF con min_df=2',
        'params': {'min_df': 2}
    },
    {
        'name': 'TF-IDF con max_df=0.8',
        'params': {'max_df': 0.8}
    },
    {
        'name': 'TF-IDF con min_df=2, max_df=0.8',
        'params': {'min_df': 2, 'max_df': 0.8}
    },
    {
        'name': 'TF-IDF con max_features=10000',
        'params': {'max_features': 10000}
    },
    {
        'name': 'TF-IDF con ngram_range=(1,2)',
        'params': {'ngram_range': (1, 2), 'max_features': 20000}
    },
    {
        'name': 'TF-IDF con ngram_range=(1,2), min_df=2, max_df=0.8',
        'params': {'ngram_range': (1, 2), 'min_df': 2, 'max_df': 0.8, 'max_features': 15000}
    },
    {
        'name': 'TF-IDF con stop_words=english',
        'params': {'stop_words': 'english'}
    },
    {
        'name': 'TF-IDF optimizado',
        'params': {'min_df': 3, 'max_df': 0.7, 'ngram_range': (1, 2), 'max_features': 12000, 'stop_words': 'english'}
    }
]

# Lista de configuraciones de modelos para probar, variando el alpha
model_configs = [
    {
        'name': 'MultinomialNB (alpha=1.0)',
        'model': MultinomialNB(alpha=1.0)
    },
    {
        'name': 'MultinomialNB (alpha=0.1)',
        'model': MultinomialNB(alpha=0.1)
    },
    {
        'name': 'MultinomialNB (alpha=0.01)',
        'model': MultinomialNB(alpha=0.01)
    },
    {
        'name': 'ComplementNB (alpha=1.0)',
        'model': ComplementNB(alpha=1.0)
    },
    {
        'name': 'ComplementNB (alpha=0.1)',
        'model': ComplementNB(alpha=0.1)
    },
    {
        'name': 'ComplementNB (alpha=0.01)',
        'model': ComplementNB(alpha=0.01)
    }
]

best_f1 = 0
best_config = None
results = []

# Probar todas las combinaciones
for vec_config in vectorizer_configs:

    vectorizer = TfidfVectorizer(**vec_config['params'])
    X_train_vec = vectorizer.fit_transform(newsgroups_train.data)
    X_test_vec = vectorizer.transform(newsgroups_test.data)
    
    for model_config in model_configs:
        model = model_config['model']
        model_name = model_config['name']
        
        # Entrenar y evaluar
        f1, _ = evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test)
        
        # Guardardo los resultados
        result = {
            'vectorizer': vec_config['name'],
            'model': model_name,
            'f1_score': f1,
            'vectorizer_params': vec_config['params'],
            'model_obj': model_config['model']
        }
        results.append(result)
        
        # Actualizo el mejor resultado si alguno tiene mejor f1 score
        if f1 > best_f1:
            best_f1 = f1
            best_config = {
                'vectorizer': vectorizer,
                'model': model_config['model'],
                'vectorizer_name': vec_config['name'],
                'model_name': model_name,
                'f1_score': f1
            }

Muestro los mejores resultados y la combinacion ganadora

In [22]:

# Ordeno por mejores resultados
results_sorted = sorted(results, key=lambda x: x['f1_score'], reverse=True)

print("Mejor configuracion de resultado")
print(f"F1-score: {best_config['f1_score']:.4f}")
print(f"Vectorizador: {best_config['vectorizer_name']}")
print(f"Modelo: {best_config['model_name']}")
print()

print("top 10 mejores configuraciones de resultados:")
print()
for i, result in enumerate(results_sorted[:10], 1):
    print(f"{i}. F1-score: {result['f1_score']:.4f}")
    print(f"   Vectorizador: {result['vectorizer']}")
    print(f"   Modelo: {result['model']}")
    print()


# Entreno con la mejor combinacion de parametros y modelo y muestro métricas adicionales
best_model = best_config['model']
best_vectorizer = best_config['vectorizer']

X_train_best = best_vectorizer.fit_transform(newsgroups_train.data)
X_test_best = best_vectorizer.transform(newsgroups_test.data)
best_model.fit(X_train_best, y_train)
y_pred_best = best_model.predict(X_test_best)

print("Resultados desgosados por clase")

print(classification_report(y_test, y_pred_best, target_names=newsgroups_train.target_names))

Mejor configuracion de resultado
F1-score: 0.6954
Vectorizador: TF-IDF default
Modelo: ComplementNB (alpha=0.1)

top 10 mejores configuraciones de resultados:

1. F1-score: 0.6954
   Vectorizador: TF-IDF default
   Modelo: ComplementNB (alpha=0.1)

2. F1-score: 0.6950
   Vectorizador: TF-IDF con max_df=0.8
   Modelo: ComplementNB (alpha=0.1)

3. F1-score: 0.6936
   Vectorizador: TF-IDF con stop_words=english
   Modelo: ComplementNB (alpha=1.0)

4. F1-score: 0.6935
   Vectorizador: TF-IDF con min_df=2
   Modelo: ComplementNB (alpha=1.0)

5. F1-score: 0.6930
   Vectorizador: TF-IDF default
   Modelo: ComplementNB (alpha=1.0)

6. F1-score: 0.6925
   Vectorizador: TF-IDF con min_df=2, max_df=0.8
   Modelo: ComplementNB (alpha=1.0)

7. F1-score: 0.6920
   Vectorizador: TF-IDF con max_df=0.8
   Modelo: ComplementNB (alpha=1.0)

8. F1-score: 0.6919
   Vectorizador: TF-IDF con stop_words=english
   Modelo: ComplementNB (alpha=0.1)

9. F1-score: 0.6907
   Vectorizador: TF-IDF con min_df=2, max_

Como puede verse, la mejor combinacion fue utilizar los parametros default del vectorizador y un modelo ComplementNB con un alpha de 0.1

## Punto 3: 
  Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [16]:
feature_names = tfidfvect.get_feature_names_out()
print(f"Numero de términos en el vocabulario: {len(feature_names)}")

# Transpongo la matriz
X_train_transposed = X_train.T 
print(f"Forma de la matriz transpuesta (término-documento): {X_train_transposed.shape}")

#Elijo manualmente 5 palabras, de distintas tematicas de los documentos
selected_words = [
    'computer',
    'baseball',
    'medical',
    'religion',
    'car'
]

# Funcion auxiliar para encontrar el índice de una palabra en el vocabulario
def find_word_index(word, feature_names):
    return list(feature_names).index(word)

# Función para encontrar las 5 palabras mas similares
def find_top_5_similar_words(word_index, X_transposed, feature_names):

    word_vector = X_transposed[word_index]
    
    similarities = cosine_similarity(word_vector, X_transposed).flatten()
    
    #Nuevamente, tomamos 6 elementos con mayor simulitud, dado que el primero sera el mismo documento (con similaridad = 1.0), y lo removemos el primero.
    most_similar_indices = similarities.argsort()[-6:][::-1]
    most_similar_indices = most_similar_indices[1:]
    
    return most_similar_indices, similarities

# Analizar cada palabra seleccionada
for word in selected_words:
    
    word_index = find_word_index(word, feature_names)
    

    word_vector = X_train_transposed[word_index]
    #Busco en cuantos docs aparece la palabra para asegurarme no haber elegido una muy inusual
    total_docs_with_word = (word_vector > 0).sum()
    print("Palabra analizada: {word}")
    print(f"{word} aparece en {total_docs_with_word} documentos")
    
    # Encontrar las 5 palabras mas similares
    similar_indices, similarities = find_top_5_similar_words(
        word_index, X_train_transposed, feature_names
    )
    
    print(f"5 palabras mas similares a '{word}':")
    print("--------------------")
    
    for rank, similar_index in enumerate(similar_indices, 1):
        similar_word = feature_names[similar_index]
        similarity_score = similarities[similar_index]
        
        # Información adicional sobre la palabra similar
        similar_vector = X_train_transposed[similar_index]
        docs_with_similar = (similar_vector > 0).sum()
        max_tfidf_similar = similar_vector.max()
        
        print(f"{rank}. '{similar_word}'")
        print(f"Similaridad coseno: {similarity_score:.4f}")
        print(f"Aparece en: {docs_with_similar} documentos")


Numero de términos en el vocabulario: 101631
Forma de la matriz transpuesta (término-documento): (101631, 11314)
Palabra analizada: {word}
computer aparece en 446 documentos
5 palabras mas similares a 'computer':
--------------------
1. 'decwriter'
Similaridad coseno: 0.1563
Aparece en: 1 documentos
2. 'harkens'
Similaridad coseno: 0.1522
Aparece en: 1 documentos
3. 'deluged'
Similaridad coseno: 0.1522
Aparece en: 1 documentos
4. 'shopper'
Similaridad coseno: 0.1443
Aparece en: 9 documentos
5. 'the'
Similaridad coseno: 0.1361
Aparece en: 9475 documentos
Palabra analizada: {word}
baseball aparece en 136 documentos
5 palabras mas similares a 'baseball':
--------------------
1. 'tommorrow'
Similaridad coseno: 0.1839
Aparece en: 6 documentos
2. 'football'
Similaridad coseno: 0.1759
Aparece en: 32 documentos
3. 'penna'
Similaridad coseno: 0.1734
Aparece en: 1 documentos
4. 'wintry'
Similaridad coseno: 0.1690
Aparece en: 1 documentos
5. 'espn'
Similaridad coseno: 0.1677
Aparece en: 39 docume

- COMPUTER:

    La mayor parte de las palabras mas similares son palabras que tienen una frecuencia de aparicion en un unico documento (decwriter, harkens y deluged). Estas palabras aparecen solo por casualidad estadistica porque por ejemplo puede aparezcan en el mismo documento, por lo que el vector seria parecido. Luego tambien puede verse el caso de 'the' que es un articulo, y tiene gran similitud simplemente porque aparece delante de la palabra computer muy seguido mas que por una similitud semantica real.

- BASEBALL:

    Sucede algo similar que lo que pasaba para 'computer' con palabras raras con alta similitud con penna y wintry en este caso, que son terminos que aparecen en un solo documento. Espn y football por otro lado si son palabras con similitudes semanticas reales. El caso de tomorrow debe darse porque es una palabra que debe aparecer en contextos similares como 'tommorrow's game' o 'baseball tomorrow'. Esto sumado a que aparece en pocos documntos probablemente genera que parezca que hay una similitud que ni hay

- MEDICAL:
    Similar a baseball, tiene una mezcla de palabras que no tienen verdadera similitud de contenido y tienen una frecuencia de aparición en muy pocos documentos con palabras que si tienen relacion entre si. Romano y relelvant aparecen en solo 1-2 documentos, osea, aparecen solo por casualidad estadística porque probablemente aparecen en el mismo documento que contiene 'medical', por lo que el vector sería parecido. Por otro lado, 'hospitals' y 'providers' si son términos médicos reales, y tienen similitud alta porque aparecen en contextos médicos similares más que por coincidencia estadística. El caso de 'recuperation' es un hibrido, dado que es una palabra que debe aparecer en contextos médicos similares como 'medical recuperation' o 'recuperation period' pero no es necesariamente tan semanticamente similar, dado que podria tambien aparecer en muchisimos otros contextos (recuperacion de un equipo en un deporte, por ejemplo)

- RELIGION:
    Sucede algo diferente que lo que pasaba para 'computer' y 'baseball'. Las palabras más similares incluyen principalmente términos religiosos reales como 'religious' y 'religions' que son derivaciones directas de la palabra original. Estas palabras tienen similitudes altas porque aparecen en contextos religiosos similares y comparten significado semántico real. El caso de 'crusades' debe darse porque es una palabra que aparece en contextos históricos-religiosos similares y ademas esta relacionado a la religion (son guerras religiosas). Solo 'purpsoe' aparece en un solo documento, mostrando que la mayoría de similitudes son semánticas reales más que coincidencias estadísticas.

- CAR:
    Sucede algo similar a 'religion' donde la mayor parte de las palabras más similares son términos automotrices reales como 'cars', 'civic', 'owner' y 'dealer'. Estas palabras tienen similitudes moderadas pero reales porque aparecen en contextos automotrices similares y comparten significado semántico en el dominio de vehículos. Solo 'criterium' aparece en un solo documento, mostrando que la mayoría de similitudes son semánticas reales más que coincidencias estadísticas. El caso de 'civic' debe darse porque es un modelo de auto específico que aparece frecuentemente en discusiones sobre autos.


De este analisis pueden destacarse dos cosas: 
- La importancia de normalizar el vocabuario para eliminar stop words o palabras demasiado infrecuentes para algunos contextos de NLP, dado podrian generar ruido a la hora de analizar similitudes en ciertos casos donde esto es contraproducente. 